In [1]:
import pickle
import torch.nn as nn
from torch import Tensor
from utils.data import Resource
import numpy as np



In [2]:
resource = Resource("../data")
data_train = resource.load_faers_train()
data_all = resource.load_faers_whole()

In [14]:
def split_train_test(data, percent_test=0.1):
    """Split the data into train/test sets.
    :param int percent_test: Percentage of data to use for testing. Default 10.
    """
    n, m = data.shape  # # users, # movies
    N = n * m  # # cells in matrix

    # Prepare train/test ndarrays.
    train = data.copy()
    test = np.ones(data.shape) * np.nan

    # Draw random sample of training data to use for testing.
    tosample = np.where(~np.isnan(train))  # ignore nan values in data
    idx_pairs = list(zip(tosample[0], tosample[1]))  # tuples of row/col index pairs

    test_size = int(len(idx_pairs) * percent_test)  # use 10% of data as test set
    train_size = len(idx_pairs) - test_size  # and remainder for training

    indices = np.arange(len(idx_pairs))  # indices of index pairs
    sample = np.random.choice(indices, replace=False, size=test_size)

    # Transfer random sample from train set to test set.
    for idx in sample:
        idx_pair = idx_pairs[idx]
        test[idx_pair] = train[idx_pair]  # transfer to test set
        train[idx_pair] = -1# remove from train set

    # Verify everything worked properly
    #assert train_size == N - np.isnan(train).sum()
    #assert test_size == N - np.isnan(test).sum()
    train[train<0.0] = np.nan
    # Return train set and test set
    return train, test

In [16]:
train , test = split_train_test(data_all*1.0)
print(train)
print(test)

[[ 1.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  1. nan  0.]
 ...
 [ 8.  0.  0. ... 10. 12.  0.]
 [ 1. nan  0. ...  4. 25.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan  8. nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan  0. nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [17]:
with open('data_all.pickle', 'wb') as handle:
    pickle.dump(data_all, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data_train.pickle', 'wb') as handle:
    pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data_test.pickle', 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:

with open('data_all.pickle', 'wb') as handle:
    pickle.dump(data_all, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data_train.pickle', 'wb') as handle:
    pickle.dump(data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)